# Orbits

In this project, we'll implement a simple physics simulation of a planet orbiting another planet. One of the key challenges in simulating physics on a computer is that our world is _continuous_, but computers can only keep track of _discrete_ pieces of information. In other words, there's no way for a computer to produce a simulation that exactly matches reality. However, in many cases, a discrete approximation is good enough to get realistic behaviour, and we'll show this is indeed the case for a simple orbiting planet example!

First, make sure you have Python 3 installed (get the latest one [here](https://www.python.org/downloads/)), and also make sure you install the following packages (we will help you do this):
* `jupyter`
* `numpy`
* `pygame`

Run all the cells to begin - you'll see that none of the planets are moving yet, and getting them to move correctly is the objective here.

In [10]:
import pygame
import numpy as np

In [11]:
parameters = {
    'G': 1,  # Gravitational constant

    # The following lists must match in size. The ith item in the lists provide parameters for the ith planet
    'init_positions': [(2.5, 0), (3, 0.5)],   # Initial positions of all moving planets, where 0,0 is the center
    'init_velocities': [(0, 2), (1.1, -0.6)],  # Initial velocities of all moving planets
    'masses': [5, 0.2],  # Masses of all moving planets

    'dt': 0.005,  # Time step, in seconds

    # Positions and masses of non-moving planets
    'anchor_planet_positions': [(0, 0)],
    'anchor_planet_masses': [10],

    'rk4_only': True   # Only shows the rk4 integrator. This hides clutter if you dont care to see the divergence
}

# Acceleration
One of the key parts to simulating physics is computing acceleration (or equivalently, forces, though we'll only work with acceleration here). In this project, we'll focus on simulating planetary orbits, which are driven by gravity.

The gravitational acceleration $\mathbf{a}$ exerted by a body of mass $M$ on another body at relative position $\mathbf{r}$ is:

$$ \mathbf{a}(\mathbf{r}) = -\frac{GM}{\|\mathbf{r}\|^2}\hat{\mathbf{r}}, $$

where $G$ is the gravitational constant, $\|\mathbf{r}\|$ is the length of $\mathbf{r}$, and $\hat{\mathbf{r}} = \frac{\mathbf{r}}{\| \mathbf{r} \|}$. This equation states that a gravitational body _pulls_ other objects towards it, with a force inversely proportional to the square of the distance to each object.

Implement this formula in the function `gravity_acceleration` below. We have provided $\| \mathbf{r} \|$ as a separate `radius` parameter.

In [12]:
def gravity_acceleration(G, M, position, radius):
    return 0 # Implement this!

# Integration

Our end goal is to know the position of our simulated planets at every point in time. Although this particular problem has a known equation for the solution (the Kepler problem), most problems in physics don't have an explicit equation describing their evolution. That's not to say we don't understand these problems at all - it just means we primarily understand them _indirectly_, through their derivatives. In fact, it's often easier to describe problems based on their derivatives. However, we're missing one more piece that will allow us to obtain trajectories from its derivatives: an _integrator_. These allow us to use position and velocity information at one point in time to make an estimate about the position and velocity at a future (nearby) point in time. In this project we'll discuss 3 different types of integrators.

## Euler Integration

The first integrator we will look at is the Euler integrator. This is conceptually the most straightforward one: we estimate how much the acceleration changes the velocity $\mathbf{v}$ during a small time interval $\Delta t$, and then use this updated velocity to estimate how much the position $\mathbf{p}$ changes over that same time interval. Mathematically it looks like this:

$$
\begin{align*}
    \mathbf{v}^{t+1} &= \mathbf{v}^t + \Delta t \mathbf{a}(\mathbf{p}^t) \\
    \mathbf{p}^{t+1} &= \mathbf{p}^t + \Delta t \mathbf{v}^{t+1} \\
\end{align*}
$$

(Note the superscripts are just to indicate the time associated with $\mathbf{p}$ and $\mathbf{v}$ and aren't exponents.)

Implement this integrator in the `euler` function below. The `acceleration` parameter is a bit strange in that it's also a function, but you can just call it like you would any other function.

In [13]:
def euler(acceleration, p, v, dt):
    # Implement this!
    v_next = v
    p_next = p
    
    return p_next, v_next

## Leapfrog Integration

Sometimes Euler integration isn't very good because it doesn't consider any acceleration values that occur _within_ the interval $\Delta t$. Depending on how much the acceleration varies in this interval, we can get trajectories that are very far off from the actual solution. To address this, we'll do something that seems a little strange: we will first take a _half-step_ to update velocity, use this half-stepped velocity to take a _full step_ in position, and then take the remaining half-step in velocity. Mathematically, it looks like this:

$$
\begin{align*}
    \mathbf{v}^{t+1/2} &= \mathbf{v}^t + \frac{\Delta t}{2} \mathbf{a}(\mathbf{p}^t) \\
    \mathbf{p}^{t+1} &= \mathbf{p}^t + \Delta t \mathbf{v}^{t+1/2} \\
    \mathbf{v}^{t+1} &= \mathbf{v}^{t+1/2} + \frac{\Delta t}{2} \mathbf{a}(\mathbf{p}^{t+1}) \\
\end{align*}
$$

Implement this integrator in the `leapfrog` function below.

In [14]:
def leapfrog(acceleration, p, v, dt):
    # Implement this!
    p_next = p
    v_next = v
    
    return p_next, v_next

## Runge-Kutta Integration

To get even more accurate results, we can take several smaller steps within the interval and use a weighted average of these substeps. Runge-Kutta integrators are a family of such methods, and here we will focus on the version that uses 4 substeps (often called RK4).

The formulas for each substep, and the final update, are as follows:

$$
\begin{align*}
    k_1 &= \Delta t \mathbf{v}^t \\
    \ell_1 &= \Delta t \mathbf{a}(\mathbf{p}^t) \\ \\
    k_2 &= \Delta t \, (\mathbf{v}^t + \ell_1/2) \\
    \ell_2 &= \Delta t \mathbf{a}(\mathbf{p}^t + k_1/2) \\ \\
    k_3 &= \Delta t \, (\mathbf{v}^t + \ell_2/2) \\
    \ell_3 &= \Delta t \mathbf{a}(\mathbf{p}^t + k_2/2) \\ \\
    k_4 &= \Delta t \, (\mathbf{v}^t + \ell_3) \\
    \ell_4 &= \Delta t \mathbf{a}(\mathbf{p}^t + k_3) \\ \\
    \mathbf{p}^{t+1} &= \mathbf{p}^t + \frac{1}{6}(k_1 + 2k_2 + 2k_3 + k_4) \\
    \mathbf{v}^{t+1} &= \mathbf{v}^t + \frac{1}{6}(\ell_1 + 2\ell_2 + 2\ell_3 + \ell_4) \\
\end{align*}
$$

This looks like a lot of math but don't worry, each section is only as complicated as the Euler method, and then we just add up the 4 results at the end.

Implement this integrator in the `rk4` function below.

In [15]:
def rk4(acceleration, p, v, dt):
    # Implement this!
    p_next = p
    v_next = v

    return p_next, v_next

## Multi-planet

As an additional challenge, write this function to compute the acceleration of a planet given a multiplanetary environment. You are provided with the gravitational constant, $G$, a list of the positions of all the other planets, and a list of corresponding masses for the other planets. Recall from physics that the net force for an object is simply the sum of all forces exerted onto that object.

The function you already implemented, `gravity_acceleration`, allows you to compute the acceleration from a force between one pair of planets. When you have multiple, what do you need to do?

As part of this function, you will need to compute the radius $||{\mathbf{r}}||$ and relative direction vector ${\mathbf{r}}$ to each planet for which you are computing the gravitational acceleration. You might need the following tools:
- `np.linalg.norm(v)` for a vector $v$ gives you the length (or magnitude) of that vector
- `v - u` for vectors $v$, $u$ gives you the vector pointing from $u$ to $v$

In [16]:
def total_acceleration(G, position: np.ndarray, planet_positions: list[np.ndarray], planet_masses: list[float]) -> np.ndarray:
    """Given a planet position and a list of positions and masses of all other planets, return the acceleration vector of the planet
    HINTS: You can add two vector positions with +, subtract them with -, or find their length with np.linalg.norm(vector)
    """
    accel = np.array([0.0, 0.0])
    for p in range(len(planet_positions)):
        # Accumulate the total acceleration in accel
        pass
    return accel

# Orbits

Now that all the integrators work, test them out! Here are a few things you can try:
* Change the `init_radius` parameter to make the orbiting planet farther away from the big planet. How does this affect the trajectories?
* Change the `init_speed` parameter to change how fast the orbiting planet is initially moving. How does this affect the trajectories?
* Change `dt` and see how effective each integrator is with different time steps.
* Try to write your own integrator! Try to come up with your own or look one up and implement that.
* Look at [Kepler's Laws of Planetary Motion](https://en.wikipedia.org/wiki/Kepler%27s_laws_of_planetary_motion) - do you observe these in your simulation? How would you verify them more rigorously?

In [17]:
class Planet:
    def __init__(self, position, velocity, parameters, method, color, is_anchored, mass):
        self.position = np.array(position)
        self.velocity = np.array(velocity)
        self.method = method
        self.is_anchored = is_anchored
        self.mass = mass

        self.setParameters(parameters)
        self.positionHistory = []
        self.color = np.array(color)
        self.secondaryColor = np.array(color)*0.8

        self.other_planets = []

    def set_other_planet_details(self, other_planets):
        self.other_planets = other_planets

    def setParameters(self, parameters):
        self.G = parameters['G']
        self.dt = parameters['dt']

    def acceleration(self, position):
        return total_acceleration(self.G, position, [p.position for p in self.other_planets], [p.mass for p in self.other_planets])

    def timestep(self):
        if not self.is_anchored:
            if self.method == 'euler':
                self.position, self.velocity = euler(self.acceleration, self.position, self.velocity, self.dt)
            elif self.method == 'leapfrog':
                self.position, self.velocity = leapfrog(self.acceleration, self.position, self.velocity, self.dt)
            elif self.method == 'rk4':
                self.position, self.velocity = rk4(self.acceleration, self.position, self.velocity, self.dt)
            else:
                raise ValueError(f'method {self.method} not recognized')
            self.positionHistory.append(self.position.copy())
    
    def Draw(self, screen, stepsPerFrame, tailSize, scale):
        x = int(self.position[0]*scale + screen.get_width()/2)
        y = int(self.position[1]*scale + screen.get_height()/2)

        # draw the orbit
        if len(self.positionHistory) > stepsPerFrame:
            steps = min(int(len(self.positionHistory) / stepsPerFrame), tailSize+1)
            end = len(self.positionHistory) - 1
            start = max(0, end - (steps-1)*stepsPerFrame)
            for i in range(start, end, stepsPerFrame):
                p1 = self.positionHistory[i]
                p2 = self.positionHistory[i+stepsPerFrame]
                p1 = (int(p1[0]*scale + screen.get_width()/2), int(p1[1]*scale + screen.get_height()/2))
                p2 = (int(p2[0]*scale + screen.get_width()/2), int(p2[1]*scale + screen.get_height()/2))
                pygame.draw.line(screen, self.secondaryColor*(i-start)/(end-start), p1, p2, 3)
        
        # draw the planet
        pygame.draw.circle(screen, self.color, (x,y), max(2 * self.mass, 3))


In [18]:
Width, Height = 1920, 1080
Width *= 0.5
Height *= 0.5
white, black = (217, 217, 217), (12, 12, 12)
size = (Width, Height)

pygame.display.init()

window = pygame.display.set_mode(size, pygame.RESIZABLE)
clock = pygame.time.Clock()
fps = 30
tailSize = 1000
dt = parameters['dt']
stepsPerFrame = max(int(1/(fps*dt)), 1)

solarSystem = []
for i in range(len(parameters['anchor_planet_positions'])):
    solarSystem.append(Planet(parameters['anchor_planet_positions'][i], (0, 0), parameters, "euler", (255, 255, 255), True, parameters['anchor_planet_masses'][i]))

eu_planets = []
leap_planets = []
rk4_planets = []

for i in range(len(parameters['init_positions'])):
    eu_planets.append(Planet(parameters['init_positions'][i], parameters['init_velocities'][i], parameters, 'euler', (255, 128, 128), False, parameters['masses'][i]))
    leap_planets.append(Planet(parameters['init_positions'][i], parameters['init_velocities'][i], parameters, 'leapfrog', (128, 255, 128), False, parameters['masses'][i]))
    rk4_planets.append(Planet(parameters['init_positions'][i], parameters['init_velocities'][i], parameters, 'rk4', (128, 128, 255), False, parameters['masses'][i]))

for i in range(len(eu_planets)):
    other_planet_idxs = [j for j in range(len(parameters['init_positions'])) if j != i]

    eu_planets[i].set_other_planet_details([eu_planets[j] for j in other_planet_idxs] + solarSystem)
    leap_planets[i].set_other_planet_details([leap_planets[j] for j in other_planet_idxs] + solarSystem)
    rk4_planets[i].set_other_planet_details([rk4_planets[j] for j in other_planet_idxs] + solarSystem)

if parameters['rk4_only']:
    solarSystem += rk4_planets
else:
    solarSystem += eu_planets + leap_planets + rk4_planets

keyPressed = False
run = True
while run:
    clock.tick(fps)
    window.fill((10, 10, 15))

    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            run = False
        if event.type == pygame.KEYDOWN:
            if event.key == pygame.K_ESCAPE:
                run = False
            keyPressed = True

	# update the window size in case it was resized
    Width, Height = window.get_size()

    for planet in solarSystem:
        for _ in range(stepsPerFrame):
            planet.timestep()
        planet.Draw(window, stepsPerFrame, tailSize, 100)

    keyPressed = False
    pygame.display.flip()

# NOTE: the window doesn't seem to close after pygame.quit() is called.
# The notebook isn't frozen and the window can be overwritten by rerunning the cells, but it's a little unintuitive.
pygame.quit()